In [45]:
import pandas as pd
import numpy as np

In [46]:
train = pd.read_csv('titanic/train.csv')

In [47]:
test = pd.read_csv('titanic/test.csv')

In [48]:
print(train.shape, test.shape) #test 데이터 프레임에는 'Survived'의 속성이 없기 때문에 11개임

(891, 12) (418, 11)


In [49]:
train_test_dataset = [train, test]

In [50]:
# data.groupby('Title')['Age'].transform('median')값을 원래는 train_title_age라는 변수로 지정해서 표기해줘야한다.
# train_title_age = data.groupby('Title')['Age'].transform('median')
# 값을 고정 안시키면 데이터 값이 바뀔 수 있기 때문에 꼭 변수를 선언해줘서 전처리 과정을 돌려야한다.

In [51]:
for data in train_test_dataset :
    data['Title'] = data['Name'].str.extract('([a-zA-Z]+)\.') 
    data['Title'] = data['Title'].str.lower()
    
    data['Title'] = np.where(data['Title'] == 'mr', 0, 
                           np.where(data['Title'] == 'miss', 1, 
                                    np.where(data['Title'] == 'mrs', 2, 3)))
    
    data['Age'] = data['Age'].fillna(data.groupby('Title')['Age'].transform('median'))
    
    data['Age'] = np.where(data['Age'] <= 16, 0, 
                             np.where(data['Age'] <= 26, 1,
                                      np.where(data['Age'] <= 36, 2,
                                               np.where(data['Age'] <= 62, 3, 4))))
    
    data['Embarked'].fillna('S', inplace=True)
    
    data['Embarked'] = np.where(data['Embarked'] == 'S', 0,
                              np.where(data['Embarked'] == 'C', 1, 2))
    
    data['Cabin'] = data['Cabin'].str[0]
    
    cabin_mapping = {'A': 0.0, 'B': 0.4, 'C': 0.8, 'D': 1.2, 'E': 1.6, 'F': 2.0, 'G': 2.4, 'T': 2.8}
    
    data['Cabin'] = data['Cabin'].map(cabin_mapping)
    data['Cabin'].fillna(data.groupby('Pclass')['Cabin'].transform('median'), inplace=True)
    
    data['Sex'] = data['Sex'].map({'male':0, 'female':1})
    
    data['Fare'].fillna(data.groupby('Pclass')['Fare'].transform('median'), inplace=True)
    data['Fare'] = np.where(data['Fare'] <= 17, 0, 
                          np.where(data['Fare'] <= 30, 1, 
                                   np.where(data['Fare'] <= 100, 2, 3)))
    
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    family_mapping = {1: 0.0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2.0, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4.0}
    data['FamilySize'] = data['FamilySize'].map(family_mapping)
    

In [52]:
features_drop = ['PassengerId', 'Name', 'Ticket', 'SibSp', 'Parch']

In [53]:
train = train.drop(features_drop, axis = 1)

In [54]:
test = test.drop(features_drop, axis = 1)

In [55]:
train.head(1)

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,0,3,0,1,0,2.0,0,0,0.4


In [56]:
test.head(1)

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,3,0,2,0,2.0,2,0,0.0


In [57]:
# train에 대한 모델링 작업

In [58]:
train.isnull().sum() #학습지, ex)모의고사

Survived      0
Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
Title         0
FamilySize    0
dtype: int64

In [59]:
test.isnull().sum() #실전, ex)수능

Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
Title         0
FamilySize    0
dtype: int64

In [ ]:
# 이렇게 하는 이유는?

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [61]:
train

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,0,3,0,1,0,2.0,0,0,0.4
1,1,1,1,3,2,0.8,1,2,0.4
2,1,3,1,1,0,2.0,0,1,0.0
3,1,1,1,2,2,0.8,0,2,0.4
4,0,3,0,2,0,2.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...
886,0,2,0,2,0,1.8,0,3,0.0
887,1,1,1,1,1,0.4,0,1,0.0
888,0,3,1,1,1,2.0,0,1,1.2
889,1,1,0,1,1,0.8,1,0,0.0


In [62]:
train.drop('Survived', axis = 1) #train.drop(columns = 'Survived')

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,3,0,1,0,2.0,0,0,0.4
1,1,1,3,2,0.8,1,2,0.4
2,3,1,1,0,2.0,0,1,0.0
3,1,1,2,2,0.8,0,2,0.4
4,3,0,2,0,2.0,0,0,0.0
...,...,...,...,...,...,...,...,...
886,2,0,2,0,1.8,0,3,0.0
887,1,1,1,1,0.4,0,1,0.0
888,3,1,1,1,2.0,0,1,1.2
889,1,0,1,1,0.8,1,0,0.0


In [63]:
inputs = train.drop('Survived', axis = 1) #train.drop(columns = 'Survived')
#문제, X

In [64]:
outputs = train['Survived']
#정답지, Y

In [65]:
print(inputs.shape, outputs.shape) #*****문제와 정답의 갯수가 일치해야 한다. *****

(891, 8) (891,)


train_input, test_input, train_output, test_output = train_test_split #학습이 잘되었는지 눈으로 확인하기 위해서 이 작업을 한다.

In [66]:
train_input, test_input, train_output, test_output = \
train_test_split(inputs, outputs, test_size = 0.2, stratify = outputs, random_state = 10)
#랜덤 값(random_state)은 고정 값(seed)을 항상 주는게 좋다.
#stratify = outputs 뜻은 train과 test 데이터의 비율을 맞추기 위함.
#순서: 1)train_input, 2)test_input, 3)train_output, 4)test_output

In [67]:
print(train_input.shape, train_output.shape)
print(test_input.shape, test_output.shape)

(712, 8) (712,)
(179, 8) (179,)


In [68]:
outputs.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [69]:
train_output.value_counts()

0    439
1    273
Name: Survived, dtype: int64

In [70]:
test_output.value_counts()

0    110
1     69
Name: Survived, dtype: int64

In [71]:
model1 = LogisticRegression()
model2 = DecisionTreeClassifier()
model3 = RandomForestClassifier()
model4 = GradientBoostingClassifier()
model5 =AdaBoostClassifier()

In [72]:
# 모델1

In [73]:
model1.fit(train_input, train_output) #input은 다 만듬. 머신러닝 모델을 만듬. (패키지를 이용해서 모델을 쉽게 만들 수 있음)

LogisticRegression()

In [74]:
model1.score(train_input, train_output) * 100 #모델1에 대한 학습데이터

82.58426966292134

In [75]:
model1.score(test_input, test_output) * 100 #

79.88826815642457

In [76]:
test_pre = model1.predict(test_input)

In [77]:
#예시
test_input.iloc[0]

Pclass        3.0
Sex           1.0
Age           0.0
Fare          1.0
Cabin         2.0
Embarked      0.0
Title         1.0
FamilySize    1.6
Name: 24, dtype: float64

In [78]:
test_pre[0] #예시에서의 예측값

0

In [79]:
test_output.iloc[0] #예시에서의 실제값

0

In [80]:
test_pre

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [81]:
accuracy_score(test_pre, test_output) * 100 #

79.88826815642457

In [82]:
# 모델2

In [83]:
model2.fit(train_input, train_output)

DecisionTreeClassifier()

In [84]:
model2.score(train_input, train_output) * 100

90.30898876404494

In [85]:
model2.score(test_input, test_output) * 100

79.3296089385475

In [86]:
test_pre = model2.predict(test_input)

In [87]:
# 모델3

In [88]:
model3.fit(train_input, train_output)

RandomForestClassifier()

In [89]:
model3.score(train_input, train_output) * 100

90.30898876404494

In [90]:
model3.score(test_input, test_output) * 100

82.68156424581005

In [91]:
test_pre = model3.predict(test_input)

In [92]:
# 모델4

In [93]:
model = model4.fit(train_input, train_output)

In [94]:
model4.score(train_input, train_output) * 100

86.65730337078652

In [95]:
model4.score(test_input, test_output) * 100

83.24022346368714

In [96]:
test_pre = model4.predict(test_input)

In [97]:
model

GradientBoostingClassifier()

In [98]:
import pickle #파이썬에 있는 기본 라이브러리이다.

In [99]:
with open('gbc.pkl', 'wb') as f : #'wb'는 write binary
    pickle.dump(model, f)

여기까지 한 수업내용이 모델을 만든 과정이다.

여기, 'rb' 구간 부터는 저장된 데이터 프레임을 들고 와서 읽는 작업을 하는 구간이다.

In [100]:
with open('gbc.pkl', 'rb') as f : #'rb'는 read binary
    load_model = pickle.load(f)

In [101]:
load_model

GradientBoostingClassifier()

In [102]:
test_input

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
24,3,1,0,1,2.0,0,1,1.6
251,3,1,2,0,2.4,0,2,0.8
694,1,0,3,1,0.8,0,3,0.0
858,3,1,1,1,2.0,1,2,1.2
784,3,0,1,0,2.0,0,0,0.0
...,...,...,...,...,...,...,...,...
839,1,0,2,1,0.8,1,0,0.0
561,3,0,3,0,2.0,0,0,0.0
808,2,0,3,0,1.8,0,0,0.0
468,3,0,2,0,2.0,2,0,0.0


In [103]:
data1 = test_input.iloc[0].to_list()

In [104]:
import numpy as np

In [105]:
np_data = np.asarray(data1)

In [106]:
re_data = np_data.reshape(1, -1)

In [107]:
pred = load_model.predict(re_data)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [108]:
if pred[0] == 0 :
    print('Died...')
else :
    print('Survived!!!')

Died...


In [109]:
# 모델5

In [110]:
model5.fit(train_input, train_output)

AdaBoostClassifier()

In [111]:
model5.score(train_input, train_output) * 100

81.32022471910112

In [112]:
model5.score(test_input, test_output) * 100

83.24022346368714

In [113]:
test_pre = model5.predict(test_input)

# 
0.전처리
1.모델 튜닝을 해야한다.
2.모델 학습시키는 곳

model = {'LogisticRegression': LogisticRegression(),
         'DecisionTreeClassifier':DecisionTreeClassifier(),
         'RandomForestClassifier': RandomForestClassifier(),
         'RandomForestClassifier': RandomForestClassifier(n_estimators = 1000, criterion = ""),
         'GradientBoostingClassifier': GradientBoostingClassifier(),
         'AdaBoostClassifier': AdaBoostClassifier() }

In [114]:
#강사님 모델링 방법

In [115]:
model = {'LogisticRegression': LogisticRegression(),
         'DecisionTreeClassifier':DecisionTreeClassifier(),
         'RandomForestClassifier': RandomForestClassifier(),
         'GradientBoostingClassifier': GradientBoostingClassifier(),
         'AdaBoostClassifier': AdaBoostClassifier() }

In [118]:
model_1 = []
train_1 = []
test_1 = []

for m in model :
    clf = model[m]
    clf.fit(train_input, train_output)
    train_score = clf.score(train_input, train_output) * 100
    test_score = clf.score(train_input, train_output) * 100
    
    print((f"[%26s] train:%.2f / test: %.2f") % (m, train_score, test_score))

[        LogisticRegression] train:82.58 / test: 82.58
[    DecisionTreeClassifier] train:90.31 / test: 90.31
[    RandomForestClassifier] train:90.31 / test: 90.31
[GradientBoostingClassifier] train:86.66 / test: 86.66
[        AdaBoostClassifier] train:81.32 / test: 81.32


In [119]:
data = list(zip(model_1, train_1, test_1))

In [126]:
df = pd.DataFrame(data, columns = ['mode1', 'train', 'test'])

In [133]:
df

,mode1,train,test


In [ ]:
df.sort_values('train', ascending = False)

,mode1,train,test


In [ ]:
# test에 대한 모델링 작업

In [ ]:
sub = pd.read_csv('titanic/submission.csv')

In [ ]:
answer = sub['Survived']

In [ ]:
test #실제 데이터 프레임을 

In [ ]:
submission = load_model.predict(test) #예측한 정답은 sub에 채웠다.

In [ ]:
submission

In [ ]:
accuracy_score(answer, submission)

In [ ]:
accuracy_score(answer, submission) * 100

In [ ]:
test.describe()

In [ ]:
# (참고)
# X AI: 설명 가능한 인공지능

머신러닝이나 AI 등은 실험적인 학문 = 경험적인 학문이다.

In [ ]:
sub2 = pd.read_csv('titanic/submission2.csv')

In [ ]:
sub2

In [ ]:
sub2['Survived'] = submission

In [ ]:
sub2

In [ ]:
sub2.to_csv('result.csv', index = False)

In [ ]:
sub2['Survived'].value_counts()